# Agents
Agents are the fundamental part of CodeGPT Plus. Here we show how to use them.

This notebooks explain how to:
- Upload a document
- Get a document
- Update document metada
- List documents
- Delete a document
- Update an agent documents

### Requirements
- Your CodeGPT Plus [API Key and Org ID](https://app.codegpt.co/es/login?next=%2Fen%2Fapikeys)
- The [Python SDK](https://github.com/JudiniLabs/judini-python): `pip install judini`

In [1]:
# SDK Installation
!pip install judini --quiet

## Initialization
To initialize CodeGPT Plus, you need to provide your API Key and Org ID. There are two ways to give this information:
1. Using environment variables `CODEGPT_API_KEY` and `CODEGPT_ORG_ID`
2. Giving the API Key and Org ID as parameters

In [4]:
from judini import CodeGPTPlus
# This will read from environment variables
codegpt_client = CodeGPTPlus()

# This explicitly sets the API key and organization ID
# import os
# CODEGPT_API_KEY = os.getenv("CODEGPT_API_KEY")
# CODEGPT_ORG_ID = os.getenv("CODEGPT_ORG_ID")
# codegpt_client = CodeGPTPlus(api_key=CODEGPT_API_KEY, org_id=CODEGPT_ORG_ID)

## Upload a document

The method `upload_document` accepts the following parameters:
- `file_path`: The path to the file to upload. This parameter is required.
- `generate_metadata`: (optional) Whether to extract metadata from the uploaded document. Default is `False`.

This method returns a dictionary with the `id` of the uploaded document. It's important to clarify that in this currently version (v0.1.10) **only `text\plain` files are supported**. Let's upload the `markdown_document.md` file.


In [8]:
document_id = codegpt_client.upload_document(file_path="./markdown_document.txt",
                                             generate_metadata=True)
document_id

Document metadata updated successfully


{'id': 'd8cf968e-764f-4143-8dcf-63ea435c6b6a'}

Let's check the document we just uploaded.

## Get a document
The method `get_document` recieve the document id and returns a `Document` object with the document information

In [10]:
my_document = codegpt_client.get_document(document_id["id"])
print(my_document.model_dump_json(indent=4))

{
    "id": "d8cf968e-764f-4143-8dcf-63ea435c6b6a",
    "user_id": "2c2bb544-2fd5-42d7-a38e-0ceafba4b062",
    "name": "markdown_document",
    "file_type": "text",
    "metadata": {
        "title": "Introduction to Markdown Syntax",
        "description": "A brief guide to Markdown, a lightweight markup language for text formatting.",
        "summary": "The document provides an overview of Markdown, explaining its plain-text formatting syntax and uses. It details how to format text, create lists, add links and images, and more.",
        "keywords": "Markdown, Text Formatting, Lists, Links, Images",
        "language": "English"
    },
    "tokens": 323,
    "chunks_count": null,
    "content": "# My Markdown Document\n\nThis is a simple example of a Markdown document. Markdown is a lightweight markup language with plain-text formatting syntax. It is often used to format readme files, documentation, and other types of text files.\n\n## Text Formatting\n\nYou can use Markdown to form

As you can note, the document includes metadata which was generated by the CodeGPTPlus API. You can update this information

## Update document metadata
The method `update_document_metadata` accepts the following parameters:
- `document_id`: The ID of the document to update. This parameter is required.
- `title`: (optional) The updated title of the document.
- `description`: (optional) The updated description of the document.
- `summary`: (optional) The updated summary of the document.
- `keywords`: (optional) The updated keywords of the document.
- `language`: (optional) The updated language of the document.

The method will update only the provided parameters and returns nothing. Let's update the title of the document.

In [11]:
codegpt_client.update_document_metadata(document_id["id"], title="My Markdown Document")

# And lets check the document again
my_document = codegpt_client.get_document(document_id["id"])
print(my_document.model_dump_json(indent=4))

Document metadata updated successfully
{
    "id": "d8cf968e-764f-4143-8dcf-63ea435c6b6a",
    "user_id": "2c2bb544-2fd5-42d7-a38e-0ceafba4b062",
    "name": "markdown_document",
    "file_type": "text",
    "metadata": {
        "title": "My Markdown Document",
        "description": "A brief guide to Markdown, a lightweight markup language for text formatting.",
        "summary": "The document provides an overview of Markdown, explaining its plain-text formatting syntax and uses. It details how to format text, create lists, add links and images, and more.",
        "keywords": "Markdown, Text Formatting, Lists, Links, Images",
        "language": "English"
    },
    "tokens": 323,
    "chunks_count": null,
    "content": "# My Markdown Document\n\nThis is a simple example of a Markdown document. Markdown is a lightweight markup language with plain-text formatting syntax. It is often used to format readme files, documentation, and other types of text files.\n\n## Text Formatting\n\n

Now, let's add a dummy document, which will be deleted later.

In [12]:
dummy_document_id = codegpt_client.upload_document(file_path="./dummy_document.txt",
                                             generate_metadata=True)
dummy_document_id

Document metadata updated successfully


{'id': 'f0318a3d-dcf2-4263-9c46-517b7b931457'}

## List all documents

We can obtain a list of all the documents by using the `get_documents` method.

In [16]:
codegpt_client.get_documents()

[Document(id='d8cf968e-764f-4143-8dcf-63ea435c6b6a', user_id='2c2bb544-2fd5-42d7-a38e-0ceafba4b062', name='markdown_document', file_type='text', metadata=DocumentMetadata(title='My Markdown Document', description='A brief guide to Markdown, a lightweight markup language for text formatting.', summary='The document provides an overview of Markdown, explaining its plain-text formatting syntax and uses. It details how to format text, create lists, add links and images, and more.', keywords='Markdown, Text Formatting, Lists, Links, Images', language='English'), tokens=323, chunks_count=None, content=None),
 Document(id='f0318a3d-dcf2-4263-9c46-517b7b931457', user_id='2c2bb544-2fd5-42d7-a38e-0ceafba4b062', name='dummy_document', file_type='text', metadata=DocumentMetadata(title='', description='', summary='', keywords='', language=''), tokens=7, chunks_count=None, content=None)]

## Delete a document

We can delete a document by using the `delete_document` method.

In [19]:
codegpt_client.delete_document(dummy_document_id["id"])

Document deleted successfully


## Update an agent documents

This is one of the most important methods. We can update the documents associated with an agent using the `update_agent_documents` method. Which receive the agent ID and a list of document IDs.

In [24]:
# Lets get the agent id first as explained in ./00_agents.ipynb
agents_list = codegpt_client.get_agents()
agent = agents_list[0] # Lets use the first agent
print(agent.model_dump_json(indent=4))

{
    "id": "b64e718d-4b7c-420b-bc5f-345991973677",
    "name": "My agent updated",
    "prompt": "You are a helpful assistant who uses a lot of emojies",
    "model": "gpt-3.5-turbo",
    "agent_documents": null,
    "welcome": "Hello, how can I help you today?",
    "pincode": null,
    "is_public": false,
    "agent_type": "qa"
}


In [23]:
# Now the list of document IDs we want to use
documents_list = codegpt_client.get_documents()
document_ids = [document.id for document in documents_list]
document_ids

['d8cf968e-764f-4143-8dcf-63ea435c6b6a']

In [25]:
codegpt_client.update_agent_documents(agent.id, document_ids)

# And now lets check the agent again
print(codegpt_client.get_agent(agent.id).model_dump_json(indent=4))

Agent documents updated successfully
{
    "id": "b64e718d-4b7c-420b-bc5f-345991973677",
    "name": "My agent updated",
    "prompt": "You are a helpful assistant who uses a lot of emojies",
    "model": "gpt-3.5-turbo",
    "agent_documents": [
        "d8cf968e-764f-4143-8dcf-63ea435c6b6a"
    ],
    "welcome": "Hello, how can I help you today?",
    "pincode": null,
    "is_public": false,
    "agent_type": "qa"
}
